* Analyze data breaches of customer/patient health information
* Analyze WIRED magazine and TechCrunch articles of the Equifax data breach 
* Dataset comes from U.S. Department of Health and Human Services Office for Civil Rights website 
* Type of data: unstructured and structured

### Which companies are the biggest offender in not securitying patients' health information? Is media coverage of a data breach always negative?

# Webscrapping Added Part 1

In [271]:
import requests
from bs4 import BeautifulSoup, SoupStrainer
from textblob import TextBlob #Natural Language Processing library 
#the pickle module implements binary protocols for serializing and de-serializing a Python object structure
import pickle

Some of this code in this step is from Alice Zhao's github account which accompanies her [youtube tutorial](https://www.youtube.com/watch?v=iQ1bfDMCv_c&feature=youtu.be) on text pre-processing for text mining

In [272]:
#a function to pull page information and return a list with multiple texts  
def url_to_transcript(url):
    page = requests.get(url).text
    content = SoupStrainer("p")#Just get <p></p> tags from website
    html_soup = BeautifulSoup(page, "html.parser", parse_only=content)
    #get the content between the <p></p> tags & put it into a list
    text = [p.get_text() for p in html_soup.find_all('p')]
    print(url)
    return text #return the list

In [273]:
#a function to combine the list of texts 
def combine_text(list_of_text):
    combined_text = ' '.join(list_of_text) #Join on whitespace
    return combined_text

<code>urls = ['https://techcrunch.com/2018/12/10/equifax-breach-preventable-house-oversight-report/',
        'https://www.wired.com/story/equifax-breach-no-excuse/']

transcripts = [url_to_transcript(u) for u in urls]#iterate over urls and makes a call to the user define function</code>

In [274]:
web_sites = ["techcrunch", "wired"]

<p>Save articles</p>
<code>for i, c in enumerate(web_sites):
    with open("transcripts/" + c + ".txt", "wb") as file:
        pickle.dump(transcripts[i], file)</code>

In [275]:
#load articles into dictionary
data = {}
for i, c in enumerate(web_sites):
    with open("transcripts/" + c + ".txt", "rb") as file:
        data[c] = pickle.load(file)

In [276]:
#combine chunks of text from same source into one text
data_combined = {key: [combine_text(value)] for (key, value) in data.items()}

In [277]:
text = []
for i in data_combined.values():
    text.append(i) #transfer dictionary values to a list

In [281]:
techcrunch = TextBlob(str(text[0]))  #for processing textual data

In [282]:
wired = TextBlob(str(text[1]))

In [283]:
wired = wired[:wired.find("WIRED is where tomorrow is realized.")] #eliminate promo sentences

# Numpy & Pandas Added Part 2

In [284]:
import pandas as pd
import numpy as np

data_breaches = pd.read_csv("F:\CIS 3120\Health Care Industry Breahes.csv")
#create datetime object for each date and place it back in DataFrame
data_breaches['Breach Submission Date'] = pd.to_datetime(data_breaches['Breach Submission Date'])
data_breaches.sort_values(by='Breach Submission Date',  ascending=True).head()

,Name of Covered Entity,State,Covered Entity Type,Individuals Affected,Breach Submission Date,Type of Breach,Location of Breached Information,Business Associate Present,Web Description
553,"Cerebral Palsy Research Foundation of Kansas, ...",KS,Healthcare Provider,8300,2018-05-08,Unauthorized Access/Disclosure,Network Server,No,NaN
552,"USACS Management Group, Ltd.",OH,Business Associate,15552,2018-05-08,Hacking/IT Incident,Email,Yes,NaN
551,Boys Town National Research Hospital,NE,NaN,2182,2018-05-09,Hacking/IT Incident,Email,Yes,NaN
550,Associates in Psychiatry and Psychology,MN,Healthcare Provider,6546,2018-05-18,Hacking/IT Incident,Network Server,No,NaN
549,Holland Eye Surgery and Laser Center,MI,Healthcare Provider,42200,2018-05-18,Hacking/IT Incident,Desktop Computer,No,NaN


In [285]:
data_breaches.shape #dimensions

(554, 9)

In [286]:
labels = ['Name of Covered Entity', 'Individuals Affected', 'Type of Breach', 'Location of Breached Information']

In [287]:
healthcare_providers = pd.DataFrame(data_breaches,columns=labels)
healthcare_providers.head()

,Name of Covered Entity,Individuals Affected,Type of Breach,Location of Breached Information
0,Ann & Robert H. Lurie Children's Hospital of C...,4824,Unauthorized Access/Disclosure,Electronic Medical Record
1,Poplar Bluff Regional Medical Center,1619,Unauthorized Access/Disclosure,Paper/Films
2,City of Detroit Health Department,1981,Unauthorized Access/Disclosure,Other
3,Ascension Eastwood Clinics,999,Unauthorized Access/Disclosure,Email
4,Agility Center Orthopedics,7000,Hacking/IT Incident,Email


In [288]:
#finds mean
print("Avg. number of people whose data was compromised: {:,.0f}".format(healthcare_providers['Individuals Affected'].mean()))

Avg. number of people whose data was compromised: 65,649


In [289]:
#finds total
print("Total number of people whose data was compromised: {:,}".format(healthcare_providers['Individuals Affected'].sum()))

Total number of people whose data was compromised: 36,369,323


In [290]:
data_breach_location = pd.Series(data_breaches['Location of Breached Information'])

In [291]:
#pandas Series unique method would still include duplicates so built loop instead
list_no_dup = [] #to hold no duplicate values 
for i, j in data_breach_location.iteritems():
    look_up = j.find(', ')#find if more than one value was passed
    if look_up > 0:
        single_location = j.split(', ')#split multiple values into a list of elements
        for k in single_location:
            if k not in list_no_dup:
                list_no_dup.append(k)#add to list if not already there
    else:
        if j not in list_no_dup:
            list_no_dup.append(j)

In [292]:
list_no_dup = np.array(list_no_dup)#to iterate faster convert to array
print("Where did the breach happen?")
for i in list_no_dup:
    print(i)

Where did the breach happen?
Electronic Medical Record
Paper/Films
Other
Email
Other Portable Electronic Device
Laptop
Network Server
Desktop Computer


# Combining & Merging Dataset Added Part 3

In [309]:
vulnerabilities_by_vendor = pd.read_csv("F:\CIS 3120\Vendor Vulnerabilities.csv", memory_map=True)
#only show results with at least 10 vulnerabilities per product
vulnerabilities_per_product = vulnerabilities_by_vendor[vulnerabilities_by_vendor['#Vulnerabilities/#Products'] >= 10]

In [294]:
vulnerabilities_per_product.head()

,Vendor Name,Number of Products,Number of Vulnerabilities,#Vulnerabilities/#Products
0,Microsoft,529,6814,13
3,Google,84,4572,54
4,Apple,119,4512,38
6,Adobe,132,3314,25
7,Debian,97,3197,33


In [295]:
vulnerabilities_by_product = pd.read_csv("F:\CIS 3120\Product Vulnerabilities.csv", memory_map=True)

In [296]:
#inner join the two csv files on Vendor Name
top_vendors = pd.merge(vulnerabilities_per_product, vulnerabilities_by_product)

In [297]:
top_vendors.head()

,Vendor Name,Number of Products,Number of Vulnerabilities,#Vulnerabilities/#Products,Product Name,Version,Number of Total Vulnerabilities
0,Microsoft,529,6814,13,Windows Server 2012,R2,940
1,Microsoft,529,6814,13,Windows Server 2012,NaN,860
2,Google,84,4572,54,Chrome,1.0.154.53,895
3,Google,84,4572,54,Chrome,1.0.154.59,892
4,Google,84,4572,54,Chrome,2.0.172.30,888


# Data Load Process Added Part 4

In [298]:
import sqlite3
conn = sqlite3.connect("F:\CIS 3120\wall_of_shame.db")#create a database
cur = conn.cursor()

In [ ]:
#transfers pandas DataFrame to database
#replace table if it already exist
data_breaches.to_sql('BREACHES', conn, if_exists='replace', index = False)

In [300]:
if (cur.connection == conn):
        conn.close()
        print("\nThe SQLite connection is closed.")


The SQLite connection is closed.


# Data Aggregation Added Part 5

In [301]:
#show total for each company
victims_by_entity = healthcare_providers.groupby("Name of Covered Entity").sum()
#show results for companies that caused at 500,000 people to have their information compromised
victims_by_entity = victims_by_entity[victims_by_entity['Individuals Affected'] >= 500000]

In [302]:
victims_by_entity

,Individuals Affected
Name of Covered Entity,
CNO ACE,566217
"Clinical Pathology Laboratories, Inc.",1733836
"Dominion Dental Services, Inc., Dominion National Insurance Company, and Dominion Dental Services USA, Inc.",2964778
Health Share of Oregon,654362
"Inmediata Health Group, Corp.",1565338
Iowa Health System d/b/a UnityPoint Health,1421107
"Optum360, LLC",11500000
UW Medicine,973024
"Women's Care Florida, LLC",528188


# Data Visualization Added Part 6

In [303]:
import scipy.special
from bokeh.io import output_file, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.layouts import row, column, gridplot
from bokeh.models.widgets import Tabs, Panel
output_notebook() # render inline in a Jupyter Notebook

Loading BokehJS ...

In [304]:
# pie chart
from math import pi
from bokeh.palettes import inferno, viridis
from bokeh.transform import cumsum

x = victims_by_entity.to_dict()['Individuals Affected'] #creates dictionary
#converts to Series with victims and company as columns
data = pd.Series(x).reset_index(name='victims').rename(columns={'index':'company'})
#creates percentage share of the pie
data['angle'] = data['victims']/data['victims'].sum() * 2*pi
#selects different color for each data point
data['color'] = inferno(len(x))

fig = figure(plot_height=350, title="Largest Healthcare Data Breaches", toolbar_location=None,
           tools="hover", tooltips="@company: @victims", x_range=(-0.5, 1.0)
           )#hover tool so name and number appear when mouse hovers over the pie chart

#creates the pie chart size and color scheme
fig.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='company', source=data)

fig.legend.location = (360,10)#location of legend
fig.axis.axis_label=None
fig.axis.visible=False
fig.grid.grid_line_color = None

# Show plot
show(fig)

In [305]:
#show results for the top 5 vendors who have at least 150 vulnerabilities per product
top_five = vulnerabilities_by_vendor[vulnerabilities_by_vendor['#Vulnerabilities/#Products'] >= 150]

In [306]:
vendor_name = top_five['Vendor Name']
risky_products = top_five['#Vulnerabilities/#Products']
#simply a mapping between column names and lists of data
source = ColumnDataSource(data=dict(company=vendor_name, counts=risky_products, color=viridis(5)))

fig = figure(x_range=vendor_name, plot_height=350, title="Vulnerabilites per product",
           toolbar_location=None, tools="")

#create bar graph
fig.vbar(x='company', top='counts', width=0.9, color='color', legend_field="company", source=source)

fig.xgrid.grid_line_color = None
fig.legend.orientation = "horizontal"
fig.legend.location = "top_center"

show(fig)

<p>In linguistics, a polarity item is a lexical item that can appear only in environments associated with a 
                 particular grammatical polarity affirmative or negative. A polarity item that appears in 
                 affirmative contexts is called a positive polarity item, and one that appears in negative 
                 contexts is a negative polarity item.</p>
<p>The polarity score is a float within the range [-1.0, 1.0]</p>

In [307]:
def sentiment_on_story(article):
    #collect the ranges of sentiment for each sentence within an article
    sentiment_scores = [round(sentence.sentiment.polarity, 4) for sentence in article.sentences]
    sentiment_category = {"negative": [], "netural": [], "positive": []}
    #places each score with its appropriate label
    for i in sentiment_scores:
        if i > 0:
            sentiment_category["positive"] += [1]
        elif i < 0:
            sentiment_category["negative"] += [1]
        else:
            sentiment_category["netural"] += [1]
    #get total for each category
    for j, k in sentiment_category.items():
        sentiment_category[j] = sum(k)
    return sentiment_category#return dictionary

In [308]:
from bokeh.palettes import brewer
x = sentiment_on_story(wired)
#converts to Series with number of sentences per emotion and sentiment as columns
data = pd.Series(x).reset_index(name='count').rename(columns={'index':'sentiment'})
#creates percentage share of the pie
data['angle'] = data['count']/data['count'].sum() * 2*pi
#selects different color for each label
data['color'] = brewer['Reds'][len(x)]

fig = figure(plot_height=350, title="Wired Magazine's coverage analyzed by sentiment", toolbar_location=None,
           tools="hover", tooltips="@sentiment: @count", x_range=(-0.5, 1.0)
           )#hover tool so name and number appear when mouse hovers over the pie chart

#creates the pie chart size and color scheme
fig.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="black", fill_color='color', legend_field='sentiment', source=data)

fig.axis.axis_label=None
fig.axis.visible=False
fig.grid.grid_line_color = None

# Show plot
show(fig)

# Advice from experts on protecting access to sensitive information 

<img src="https://media.giphy.com/media/MGaacoiAlAti0/giphy.gif" alt="Drawing" style="width: 500px;"/>

* Apply all available security updates for VPN and firewall configurations.
* Monitor and pay special attention to your remote access infrastructure. 
* Turn on attack surface reduction rules, including rules that block credential theft and ransomware activity. 
* Turn on AMSI for Office VBA if you have Office 365.
* Harden internet-facing assets and ensure that they have the latest security updates.
* Secure your Remote Desktop Gateway using solutions like Azure Multi-Factor Authentication (MFA). If you don't have an MFA gateway, enable network-level authentication (NLA).
* Practice the principle of least-privilege and maintain credential hygiene.
* Monitor for brute-force attempts.
* Monitor for the clearing of Event Logs, especially the Security Event log and PowerShell Operational logs.
* Determine where highly privileged accounts are logging on and exposing credentials.
* Use the Windows Defender Firewall and your network firewall to prevent Remote Procedure Call (RPC) and Server Message Block (SMB) communication among endpoints whenever possible. This limits lateral movement as well as other attack activities.

# Citations & References:
* [U.S. Department of Health and Human Services
Office for Civil Rights](https://ocrportal.hhs.gov/ocr/breach/breach_report.jsf)
* [What is the HITECH ACT?](https://compliancy-group.com/what-is-the-hitech-act/)
* [
Microsoft to hospitals: 11 tips on how to combat ransomware
](https://www.techrepublic.com/article/microsoft-to-hospitals-11-tips-on-how-to-combat-ransomware/)
* [What is HIPAA Compliance?](https://compliancy-group.com/hipaa/)
* [Top 50 Vendors By Total Number Of "Distinct" Vulnerabilities](https://www.cvedetails.com/top-50-vendors.php)
* [Top 50 Versions Of Products Having Highest Number Of Security Vulnerabilities Related To Them](https://www.cvedetails.com/top-50-versions.php)
* [SQLite3 Datatypes](https://www.sqlite.org/datatype3.html)
* [SQLite3 Python documentation](https://docs.python.org/3/library/sqlite3.html)
* [
Data Cleaning by Alice Zhao](https://github.com/adashofdata/nlp-in-python-tutorial/blob/master/1-Data-Cleaning.ipynb)
* [textblob](https://textblob.readthedocs.io/en/dev/)
* [Bokeh pie chart](https://docs.bokeh.org/en/1.4.0/docs/gallery/pie_chart.html)
* [Bokeh bar graph](https://docs.bokeh.org/en/1.4.0/docs/gallery/bar_colors.html)